In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [168]:
data = pd.read_csv('train.csv')
x = data.iloc[:,[2,4,5,6,7,9,11]]
y = data.iloc[:,1]

In [169]:
from sklearn.impute import SimpleImputer
categorical_features = x.select_dtypes(include=['object','category']).columns
imputer = SimpleImputer(strategy='most_frequent')
x[categorical_features] = imputer.fit_transform(x[categorical_features])

/var/folders/8_/kb8645j910n7lcgmfs8c0d0m0000gn/T/ipykernel_4060/2633556173.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[categorical_features] = imputer.fit_transform(x[categorical_features])


In [170]:
imputer = SimpleImputer(strategy='mean')
x['Age'] = imputer.fit_transform(x[['Age']])

/var/folders/8_/kb8645j910n7lcgmfs8c0d0m0000gn/T/ipykernel_4060/3436635348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Age'] = imputer.fit_transform(x[['Age']])


In [171]:
x = pd.get_dummies(x,['Sex','Embarked']).astype(int)